In [1]:
# specify the aws profile and region you want to use to deploy the stack
aws_profile="bbdev"
region_name="us-east-2"
binary_name="lambda"

# Create AWS Lambda function

Create a terraform container containing the binary to be deployed and the infra script, with `terraform init` already executed.

In [ ]:
%%bash -s "$binary_name"
date
DOCKER_BUILDKIT=1 docker build -q \
    -f Dockerfile \
    --build-arg BIN_NAME=$1 \
    --target terraform \
    -t cloudfuse/cloud-reader-terraform \
    ..

In [ ]:
%%bash -s "$aws_profile" "$region_name" "$binary_name"
date
docker run \
    --rm \
    -v $HOME/.aws/credentials:/creds:ro \
    -v cloud-reader-tf:/mnt/state_vol \
    cloudfuse/cloud-reader-terraform apply -var="profile=$1" -var="region_name=$2" -var="binary_name=$3" -auto-approve

# Destroy AWS Lambda function

In [6]:
%%bash -s "$aws_profile" "$region_name" "$binary_name"
date
docker run \
    --rm \
    -v $HOME/.aws/credentials:/creds:ro \
    -v cloud-reader-tf:/mnt/state_vol \
    cloudfuse/cloud-reader-terraform destroy -var="profile=$1" -var="region_name=$2" -var="binary_name=$3" -auto-approve

Fri Jul 30 22:22:33 CEST 2021
module.lambda_function.aws_cloudwatch_log_group.lambda[0]: Refreshing state... [id=/aws/lambda/cloud-reader-benchmark]
module.lambda_function.aws_iam_role.lambda[0]: Refreshing state... [id=cloud-reader-benchmark]
module.lambda_function.aws_iam_policy.logs[0]: Refreshing state... [id=arn:aws:iam::615900053518:policy/cloud-reader-benchmark-logs]
module.lambda_function.aws_lambda_function.this[0]: Refreshing state... [id=cloud-reader-benchmark]
module.lambda_function.aws_iam_policy_attachment.logs[0]: Refreshing state... [id=cloud-reader-benchmark-logs]

Terraform used the selected providers to generate the following execution
plan. Resource actions are indicated with the following symbols:
  - destroy

Terraform will perform the following actions:

  # module.lambda_function.aws_cloudwatch_log_group.lambda[0] will be destroyed
  - resource "aws_cloudwatch_log_group" "lambda" {
      - arn               = "arn:aws:logs:us-east-2:615900053518:log-group:/aws/l

In [7]:
%%bash
date
docker volume rm cloud-reader-tf

Fri Jul 30 22:22:53 CEST 2021
cloud-reader-tf


In [8]:
%%bash
date
docker image rm cloudfuse/cloud-reader-terraform

Fri Jul 30 22:22:54 CEST 2021
Untagged: cloudfuse/cloud-reader-terraform:latest
Deleted: sha256:4296d4a281223222b34a0112512c7d9cad61669e1365a93336f9f87eef172588
